# Preâmbulo

Imports, funções e downloads.

In [0]:
# Basic imports.
import os
import time
import numpy as np
import torch

from torch import nn
from torch import optim

from torch.utils.data import DataLoader
from torch.utils import data
from torch.backends import cudnn

from torchvision import models
from torchvision import datasets
from torchvision import transforms

from skimage import io

from sklearn import metrics

from matplotlib import pyplot as plt

%matplotlib inline

cudnn.benchmark = True

In [2]:
# Setting predefined arguments.
args = {
    'epoch_num': 50,      # Number of epochs.
    'n_classes': 10,      # Number of classes.
    'lr': 1e-4,           # Learning rate.
    'weight_decay': 5e-4, # L2 penalty.
    'momentum': 0.9,      # Momentum.
    'num_workers': 3,     # Number of workers on data loader.
    'batch_size': 50,     # Mini-batch size.
    'w_size': 224,        # Width size for image resizing.
    'h_size': 224,        # Height size for image resizing.
}

if torch.cuda.is_available():
    args['device'] = torch.device('cuda')
else:
    args['device'] = torch.device('cpu')

print(args['device'])

cuda


# DenseNets

As [Densely Connected Convolutional Networks](https://arxiv.org/abs/1608.06993) (DenseNets) surgiram em 2016 como um avanço em relação ao antigo estado da arte: as [ResNets](https://arxiv.org/abs/1512.03385). Essas redes, ao invés de focarem em estabilizar o treinamento para conseguir arquiteturas cada vez mais profundas, tinham em vista obter uma melhor **eficiência de parâmetros** com relação às suas antecessoras. Vários artigos entre 2015 e 2016 observaram que, apesar de serem altamente poderosas, as ResNets possuiam vários canais completamente redundantes que poderiam ser simplesmente ignorados no forward durante o teste sem afetar o desempenho final da rede. Isso indicava que essas redes possuiam uma baixa eficiência de parâmetros.

As DenseNets surgiram como uma forma de mitigar esse problema dos canais convolucionais redundantes, mas ainda assim mantendo a característica da função identidade das ResNets que funciona essencialmente como uma **Skip Connection** para mitigar o problema do **Vanishing Gradient** durante o backpropagation. Enquanto ResNets usam uma soma como função de identidade, como vimos no começo da semana, DenseNets propuseram o uso de concatenações como funções identidades, assim como FCNs fazem. Porém, ao contrário das FCNs, cada bloco convolucional de uma DenseNet pode receber vários feature maps, mas quase sempre vai combinar esses canais de entrada em apenas alguns poucos canais de saída, efetivamente economizando parâmetros a serem computados. Abaixo pode ser visto um exemplo de um **Bloco Convolucional Denso**, como pode ser visto na figura abaixo.

![Dense Block](https://www.dropbox.com/s/xmpfsei3rdyx2s5/DenseNets_Dense_Block.png?dl=1)

Assim como no caso dos **Blocos Residuais** das ResNets, vários **Blocos Convolucionais Densos** são empilhados para formar uma DenseNet. Entre esses blocos são colocadas **Camadas de Transição** compostas simplesmente por convoluções 1x1 e average pooling. As **Camadas de Transição** servem para diminuir progressivamente as resoluções espaciais dos feature maps, as quais são preservadas dentro de cada **Bloco Denso**. **Camadas de Transição** também controlam a quantidade de feature maps que passam para o próximo **Bloco Denso** por meio da convolução 1x1. Por fim, assim como em praticamente todas as CNNs para classificação modernas (i.e. Inception, ResNets...), os canais que saem do último **Bloco Denso** são linearizados usando um **Global Average Pooling** e passados para uma camada **Fully Connected** que realiza a predição por classes.

![DenseNet Architecture](https://www.dropbox.com/s/xj4lz2ykv977p6f/DenseNets_Architecture.png?dl=1)

DenseNets se provaram muito mais eficientes em número de parâmetros e em [flops](https://pt.wikipedia.org/wiki/FLOPS), quando comparadas com as ResNets, como pode ser visto nos gráficos abaixo.

![DenseNet vs. ResNet 1](https://www.dropbox.com/s/zdgzwh3hciex93y/DenseNets_vs_ResNets_1.png?dl=1)
![DenseNet vs. ResNet 2](https://www.dropbox.com/s/c6oz6xpu3bkt306/DenseNets_vs_ResNets_2.png?dl=1)
![DenseNet vs. ResNet 3](https://www.dropbox.com/s/dhageb200bogvds/DenseNets_vs_ResNets_3.png?dl=1)
![DenseNet vs. ResNet 4](https://www.dropbox.com/s/flt39pmiy3bophr/DenseNets_vs_ResNets_4.png?dl=1)

Comparando o desempenho de DenseNets em vários datasets, é visível que as DenseNets conseguem resultados melhores que todas as concorrentes com um número de parâmetros treináveis relativamente modesto.

![DenseNet vs. Others](https://www.dropbox.com/s/en016pm6kocwj7m/DenseNets_vs_Other_Networks.png?dl=1)

# Atividade Prática: Aproveitando DenseNet pré-treinada do Pytorch

Aproveitar o resto dos códigos da S04A04_1 e adaptá-los para realizar classificação no CIFAR 10 usando a DenseNet121 pré-treinada do subpacote [*models*](https://pytorch.org/docs/stable/torchvision/models.html) do torchvision.

In [0]:
# Using predefined and pretrained model of DenseNet121 on torchvision.
net = # ...